In [ ]:
# Inital state probability matrix, given
PI = [0.5,0.5]
# state transition probability matrix, given
A = [[0.8,0.2],[0.001,0.999]]
# Emission or observation probability matrix, given
B = [[0.5,0.1,0.3,0.1,0.0],[0.05,0.35,0.2,0.2,0.2]]
# state codes of the observed states, dummy
Observables = [0,1,2,3,4]
# Observables = {"Movement":0, "Passive Social":1, "Active Social":2, "Texting":3, "Acess Psych Site":4}

In [ ]:
def filtering(PI, A, B, Observables):
    return #alpha (normalized forward probability matrix), c (one-dimensional normalization (scaling) coefficient array)

In [ ]:
def evaluation(alpha):
    return #probability

In [ ]:
def backward(PI, A, B, Observables):
    return #no of initial states * length of observation sequence